 # Análisis Exploratorio de Datos con Python

In [42]:
import pandas as pd
import numpy as np

# Cargar los tres datasets tratados:

In [40]:

try:
    df_tratado_caracteristicas_equipos = pd.read_csv('../output/df_caracteristicas_equipos_tratados.csv')
    df_tratado_historico_ordenes = pd.read_csv('../output/df_historico_ordenes_tratados.csv')
    df_tratado_registros_condiciones = pd.read_csv('../output/df_registros_condiciones_tratados.csv')

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")

print (df_tratado_caracteristicas_equipos.info())
print (df_tratado_historico_ordenes.info())
print (df_tratado_registros_condiciones.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID_Equipo                    500 non-null    int64  
 1   Tipo_Equipo                  500 non-null    object 
 2   Fabricante                   500 non-null    object 
 3   Modelo                       500 non-null    object 
 4   Potencia_kW                  500 non-null    float64
 5   Horas_Recomendadas_Revision  500 non-null    int64  
 6   Frecuencia_Mantenimiento     500 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9951 entries, 0 to 9950
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_Orden             9951 non-null   int64  
 1   ID_Equipo            9951 non-null   int64  


# Estadísticas básicas como la media, mediana y desviación estándar de las horas operativas, el costo de mantenimiento, y la frecuencia de fallos.

In [53]:
def calcular_estadisticas_por_equipo(df_caracteristicas, df_historico, df_condiciones, columna_equipo='ID_Equipo'):

    estadisticas_equipos = []

    equipos_unicos = df_caracteristicas[columna_equipo].unique()

    
    for equipo in equipos_unicos:
        equipo_stats = {'ID_Equipo': equipo}

        df_equipo_carac = df_caracteristicas[df_caracteristicas[columna_equipo] == equipo]
        df_equipo_hist = df_historico[df_historico[columna_equipo] == equipo]
        df_equipo_cond = df_condiciones[df_condiciones[columna_equipo] == equipo]

        if df_equipo_cond.empty:
            print(f"⚠️ No hay datos en df_condiciones para el equipo {equipo}")


        if 'Horas_Operativas' in df_equipo_cond.columns and not df_equipo_cond.empty:
            horas_operativas = df_equipo_cond['Horas_Operativas'].dropna()
            if not horas_operativas.empty:
                equipo_stats['horas_operativas_media'] = horas_operativas.mean()
                equipo_stats['horas_operativas_mediana'] = horas_operativas.median()
                equipo_stats['horas_operativas_std'] = horas_operativas.std()
            else:
                print(f"⚠️ 'Horas_Operativas' tiene solo valores nulos para el equipo {equipo}")
        else:
            print(f"⚠️ 'Horas_Operativas' no encontrada para el equipo {equipo}")


        if 'Costo_Mantenimiento' in df_equipo_hist.columns and not df_equipo_hist.empty:
            costo_mant = df_equipo_hist['Costo_Mantenimiento'].dropna()
            if not costo_mant.empty:
                equipo_stats['costo_mantenimiento_media'] = costo_mant.mean()
                equipo_stats['costo_mantenimiento_mediana'] = costo_mant.median()
                equipo_stats['costo_mantenimiento_std'] = costo_mant.std()
            else:
                print(f"⚠️ 'Costo_Mantenimiento' tiene solo valores nulos para el equipo {equipo}")


        if 'Frecuencia_Mantenimiento' in df_equipo_carac.columns and not df_equipo_carac.empty:
            frecuencia_fallos = df_equipo_carac['Frecuencia_Mantenimiento'].dropna()
            if not frecuencia_fallos.empty:
                equipo_stats['frecuencia_fallos_media'] = frecuencia_fallos.mean()
                equipo_stats['frecuencia_fallos_mediana'] = frecuencia_fallos.median()
                equipo_stats['frecuencia_fallos_std'] = frecuencia_fallos.std()
            else:
                print(f"⚠️ 'Frecuencia_Mantenimiento' tiene solo valores nulos para el equipo {equipo}")

        estadisticas_equipos.append(equipo_stats)


    df_estadisticas = pd.DataFrame(estadisticas_equipos)

    return df_estadisticas

df_resultado = calcular_estadisticas_por_equipo(df_tratado_caracteristicas_equipos, 
                                                df_tratado_historico_ordenes, 
                                                df_tratado_registros_condiciones)

print(df_resultado)

⚠️ No hay datos en df_condiciones para el equipo 500
⚠️ 'Horas_Operativas' no encontrada para el equipo 500
     ID_Equipo  horas_operativas_media  horas_operativas_mediana  \
0            1            58004.800000                   60480.0   
1            2            55557.590909                   57857.5   
2            3            60570.473684                   64675.0   
3            4            57826.571429                   61294.0   
4            5            46757.714286                   45693.5   
..         ...                     ...                       ...   
495        496            32025.111111                   17753.0   
496        497            42631.888889                   55874.0   
497        498            43099.600000                   44334.0   
498        499            45653.400000                   46817.0   
499        500                     NaN                       NaN   

     horas_operativas_std  costo_mantenimiento_media  \
0            33710.

In [54]:
def detectar_outliers_zscore(df, columna, umbral=3):

    media = df[columna].mean()
    std = df[columna].std()
    
    df["Z_Score"] = np.abs((df[columna] - media) / std)
    outliers = df[df["Z_Score"] > umbral]
    
    return outliers.drop(columns=["Z_Score"])


In [47]:
detectar_outliers_zscore(df_tratado_historico_ordenes, "Costo_Mantenimiento", 2)

,ID_Orden,ID_Equipo,Fecha,Tipo_Mantenimiento,Costo_Mantenimiento,Duracion_Horas,Ubicacion
172,173,481,2020-01-08 04:00:00,Correctivo,59674.687521,20,Planta Sur
383,385,131,2020-01-17 00:00:00,Preventivo,67082.733179,24,Planta Este
423,425,221,2020-01-18 16:00:00,Preventivo,71857.600220,34,Planta Norte
582,585,202,2020-01-25 08:00:00,Preventivo,73048.320722,2,Planta Norte
636,639,187,2020-01-27 14:00:00,Preventivo,79265.076671,27,Planta Este
1867,1876,291,2020-03-19 03:00:00,Correctivo,86545.216355,44,Planta Oeste
2070,2079,239,2020-03-27 14:00:00,Correctivo,86861.031776,20,Planta Este
2506,2520,259,2020-04-14 23:00:00,Correctivo,79846.188496,26,Planta Norte
2578,2592,264,2020-04-17 23:00:00,Preventivo,80366.318975,3,Planta Este
2709,2723,39,2020-04-23 10:00:00,Correctivo,94440.350492,44,Planta Sur


In [48]:
detectar_outliers_zscore(df_tratado_registros_condiciones, "Horas_Operativas", 1)


,ID_Registro,ID_Equipo,Fecha,Temperatura_C,Vibracion_mm_s,Horas_Operativas
5,6,48,2020-01-01 05:00:00,98.77,2.73,89664.0
7,8,151,2020-01-01 07:00:00,113.15,2.11,90374.0
8,9,280,2020-01-01 08:00:00,63.39,2.78,79244.0
9,10,224,2020-01-01 09:00:00,122.37,1.91,91137.0
10,11,273,2020-01-01 10:00:00,59.49,4.64,18341.0
...,...,...,...,...,...,...
8953,8994,408,2021-01-09 17:00:00,75.97,1.23,82000.0
8955,8996,299,2021-01-09 19:00:00,51.08,2.32,97743.0
8956,8997,160,2021-01-09 20:00:00,122.18,2.65,84374.0
8957,8998,160,2021-01-09 21:00:00,119.70,2.98,16939.0


In [56]:
def analizar_distribucion_fallos(df):
    # Contar la cantidad de mantenimientos por tipo
    distribucion_tipo_mantenimiento = df["Tipo_Mantenimiento"].value_counts().reset_index()
    distribucion_tipo_mantenimiento.columns = ["Tipo_Mantenimiento", "Cantidad"]
    
    # Contar la cantidad de mantenimientos por equipo
    fallos_por_equipo = df.groupby("ID_Equipo")["ID_Orden"].count().reset_index()
    fallos_por_equipo.columns = ["ID_Equipo", "Cantidad_Fallos"]
    
    # Resumen de costos promedio por tipo de mantenimiento
    costo_promedio_mantenimiento = df.groupby("Tipo_Mantenimiento")["Costo_Mantenimiento"].mean().reset_index()
    costo_promedio_mantenimiento.columns = ["Tipo_Mantenimiento", "Costo_Promedio"]
    
    # Unir toda la información en un solo dataframe resumen
    resumen = distribucion_tipo_mantenimiento.merge(
        costo_promedio_mantenimiento, on="Tipo_Mantenimiento", how="left"
    )
    
    return distribucion_tipo_mantenimiento, fallos_por_equipo, resumen

# Llamar a la función con tu DataFrame
distribucion_tipo_mantenimiento, fallos_por_equipo, resumen = analizar_distribucion_fallos(df_tratado_historico_ordenes)


In [59]:
print (distribucion_tipo_mantenimiento)
print (fallos_por_equipo)
print (resumen)

  Tipo_Mantenimiento  Cantidad
0         Preventivo      5000
1         Correctivo      4951
     ID_Equipo  Cantidad_Fallos
0            1               22
1            2               24
2            3               12
3            4               21
4            5               18
..         ...              ...
494        495               20
495        496               16
496        497               23
497        498               21
498        499               15

[499 rows x 2 columns]
  Tipo_Mantenimiento  Cantidad  Costo_Promedio
0         Preventivo      5000     5263.285618
1         Correctivo      4951     5248.322251


In [60]:
# Calcular la matriz de correlación
correlation_matrix = df_tratado_registros_condiciones[["Temperatura_C", "Vibracion_mm_s", "Horas_Operativas"]].corr()

# Mostrar la matriz de correlación
print(correlation_matrix)

                  Temperatura_C  Vibracion_mm_s  Horas_Operativas
Temperatura_C          1.000000       -0.001674          0.009831
Vibracion_mm_s        -0.001674        1.000000          0.003989
Horas_Operativas       0.009831        0.003989          1.000000
